In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb

In [2]:
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier, DMatrix, cv
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc


In [3]:
#Fetching the pre processed dataset
columns=['µStrain', 'µStrain.1', 'µStrain.2', 'µStrain.3', 'µStrain.4', 'µStrain.5', 'µStrain.6', 'µStrain.7', 'µStrain.8', 'µStrain.9', 'µStrain.10', 'µStrain.11', 'µStrain.12', 'µStrain.13', 'µStrain.14', 'µStrain.15', 'µStrain.16', 'µStrain.17', 'µStrain.18', 'µStrain.19', 'µStrain.20', 'µStrain.21', 'µStrain.22', 'µStrain.23', 'µStrain.24', 'µStrain.25', 'µStrain.26', 'µStrain.27', 'µStrain.28', 'µStrain.29', 'µStrain.30', 'µStrain.31', 'µStrain.32', 'µStrain.33', 'µStrain.34', 'µStrain.35', 'µStrain.36', 'µStrain.37', 'µStrain.38', 'µStrain.39', 'µStrain.40', 'µStrain.41', 'µStrain.42', 'µStrain.43', 'µStrain.44', 'µStrain.45', 'µStrain.46', 'µStrain.47', 'µStrain.48', 'µStrain.49', 'µStrain.50', 'µStrain.51', 'µStrain.52', 'µStrain.53','µStrain.54','µStrain.55','µStrain.56','µStrain.57','µStrain.58','µStrain.59','µStrain.60','µStrain.61', 'Target']
df = pd.read_parquet('/Users/piyush/StudienArbeit/df_u.parquet',columns=columns) 
df = df.dropna()
values = ['0.0-_LL-FXXf', '0.6-_LL-FXXf', '0.2-_LL-FXXf', '0.4-_LL-FXXf', '0.5-_LL-FXXf', '0.65-_LL-FXXf', '0.7-_LL-FXXf', '0.75-_LL-FXXf' 
, '0.0-_LL-LBLf', '0.6-_LL-LBLf', '0.2-_LL-LBLf', '0.4-_LL-LBLf', '0.5-_LL-LBLf', '0.65-_LL-LBLf', '0.7-_LL-LBLf', '0.75-_LL-LBLf'
, '0.0-_LL-VBDf', '0.6-_LL-VBDf', '0.2-_LL-VBDf', '0.4-_LL-VBDf', '0.5-_LL-VBDf', '0.65-_LL-VBDf', '0.7-_LL-VBDf', '0.75-_LL-VBDf']
df = df[df.Target.isin(values) == True]


print('Shape of the dataset:' +str(df.shape) )
print(df.head(-5))
print(df['Target'].unique())

Shape of the dataset:(4239538, 63)
         µStrain  µStrain.1  µStrain.2  µStrain.3  µStrain.4  µStrain.5  \
0          399.5      384.2      541.4      500.2      305.5      315.7   
1          402.6      386.2      544.2      502.2      305.8      317.1   
2          400.5      385.9      541.4      502.3      304.6      315.9   
3          400.6      384.9      541.5      501.6      304.0      316.2   
4          401.5      385.2      541.4      502.1      303.8      316.6   
...          ...        ...        ...        ...        ...        ...   
4240273    335.2      320.2      486.5      431.1      279.5      267.5   
4240274    335.2      320.2      486.5      431.1      279.5      267.5   
4240275    335.7      319.5      486.5      431.3      278.4      267.1   
4240276    335.7      319.5      486.5      431.3      278.4      267.1   
4240277    335.7      319.5      486.5      431.3      278.4      267.1   

         µStrain.6  µStrain.7  µStrain.8  µStrain.9  ...  µStrai

In [4]:
X = df.iloc[:, 0:61].values
y = df.iloc[: , 62].values
print(y)
le = LabelEncoder()
y_enc = le.fit_transform(y)


print(X)
print(y_enc)


['0.5-_LL-LBLf' '0.5-_LL-LBLf' '0.5-_LL-LBLf' ... '0.2-_LL-LBLf'
 '0.2-_LL-LBLf' '0.2-_LL-LBLf']
[[399.5 384.2 541.4 ...  34.5  14.7  29.7]
 [402.6 386.2 544.2 ...  34.6  14.7  29.3]
 [400.5 385.9 541.4 ...  34.9  14.2  28.2]
 ...
 [324.9 309.9 472.2 ...  61.3  19.3  38.2]
 [324.9 309.9 472.2 ...  61.3  19.3  38.2]
 [322.7 307.8 470.  ...  60.6  19.2  38.1]]
[10 10 10 ...  4  4  4]


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y_enc, test_size=0.3, random_state=42)
print ('Feature Training Set:', X_train.shape)
print('Feature Testing Set:', X_test.shape)
print('Label Trainhing Set:', y_train.shape)
print('Label Testing Set:', y_test.shape)


Feature Training Set: (2967676, 61)
Feature Testing Set: (1271862, 61)
Label Trainhing Set: (2967676,)
Label Testing Set: (1271862,)


In [6]:
# Create classification matrices
dtrain_clf = DMatrix(X_train, y_train, enable_categorical='True')
dtest_clf = DMatrix(X_test, y_test, enable_categorical='True')


In [7]:
#Creating a classifier
clf = XGBClassifier(objective='multi:softprob', nthread=4, seed=42, n_estimators = 100, max_depth = 4, max_delta_step = 1, subsample = 0.5, eta=0.1)
clf.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.1, eval_metric=None,
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=1, max_depth=4,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              nthread=4, num_parallel_tree=None, ...)

In [9]:
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


Accuracy: 0.9999968550047096


In [14]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


ValueError: multiclass format is not supported